In [1]:
from sklearn.neighbors import NearestNeighbors

import pandas as pd

import os

os.chdir('..')

In [2]:
data_path = r'data/dummy-data.csv'

queries_path = r'data/dummy-queries.csv'

In [3]:
data = pd.read_csv(data_path).astype(float)

In [4]:
queries = pd.read_csv(queries_path).astype(float)

In [5]:
def get_subset(data, query_type, v, l, r):

    if query_type == 1.0:

        return data[data['C'] == v]
    
    elif query_type == 2.0:

        return data[(data['T'] >= l) & (data['T'] <= r)]
    
    elif query_type == 3.0:

        return data[(data['C'] == v) & ((data['T'] >= l) & (data['T'] <= r))]
    
    return data

In [6]:
def conditional_brute_force_knn_search(data, queries):
    k = 100
    results = []
    
    prev_subset = None
    prev_query_type, prev_v, prev_l, prev_r = None, None, None, None
    knn_model = None

    for index, row in queries.iterrows():

        query_type = row['query_type']
        v = row['v']
        l = row['l']
        r = row['r']
        query_id = row['vector']
        query_vector = row.filter(like='f').values
        
        if (query_type != prev_query_type) or (v != prev_v) or (l != prev_l) or (r != prev_r):
            subset = get_subset(data, query_type, v, l, r)
            knn_model = NearestNeighbors(n_neighbors=k, algorithm='brute')
            knn_model.fit(subset.filter(like='f').values)

            prev_subset = subset

            prev_query_type, prev_v, prev_l, prev_r = query_type, v, l, r

        else:

            subset = prev_subset

        distances, data_indexes = knn_model.kneighbors(query_vector.reshape(1, -1))

        for loop, d_i in enumerate(data_indexes.flatten()):

            result = {
                'query_vector': query_id,
                'Distance': distances.flatten()[loop],
                'data_vector': subset.iloc[d_i]['vector'],
                'query_type': query_type,
                'C': subset.iloc[d_i]['C'],
                'v': v,
                'l': l,
                'T': subset.iloc[d_i]['T'],
                'r': r
                }

            results.append(result)

    results_df = pd.DataFrame(results)

    return results_df

In [7]:
res = conditional_brute_force_knn_search(data, queries)

res.to_parquet('output/BRUTE_FORCE.parquet')